In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
import gc
import os
import matplotlib.pyplot as plt
import seaborn as sns 
import lightgbm as lgb
import itertools
import pickle, gzip
import glob
from sklearn.preprocessing import StandardScaler
from tsfresh.feature_extraction import extract_features
np.warnings.filterwarnings('ignore')
import dask.dataframe as dd
import missingno as msno
from pandasql import sqldf
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.model_selection import KFold
import matplotlib.gridspec as gridspec
from sklearn import preprocessing
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

In [2]:
#Always seed the randomness of this universe
np.random.seed(51)

In [3]:
%%time
pred1 = dd.read_csv('test_pred_20.csv')
pred2 = dd.read_csv('test_pred_22.csv')
pred1 = pred1.compute()
pred2 = pred2.compute()
print(pred1.shape,pred2.shape)

(3492890, 16) (3492890, 16)
CPU times: user 33.7 s, sys: 3.26 s, total: 36.9 s
Wall time: 7.81 s


In [4]:
%%time
pred1 = pred1.reset_index(drop=True)
pred2 = pred2.reset_index(drop=True)

CPU times: user 201 ms, sys: 269 ms, total: 470 ms
Wall time: 468 ms


In [5]:
pred1['object_id'].equals(pred2['object_id'])

True

In [6]:
%%time
test_ensemble = pd.DataFrame()

CPU times: user 273 µs, sys: 35 µs, total: 308 µs
Wall time: 312 µs


In [7]:
coef_lgb = 0.5
coef_mlp = 0.5

In [8]:
%%time
test_ensemble = pred1.iloc[:,1:]*coef_lgb + pred2.iloc[:,1:]*coef_mlp

CPU times: user 370 ms, sys: 609 ms, total: 979 ms
Wall time: 977 ms


In [9]:
print(test_ensemble.shape)

(3492890, 15)


In [10]:
test_ensemble['object_id'] = pred1['object_id']

In [11]:
print(test_ensemble.shape)

(3492890, 16)


In [12]:
test_ensemble.tail()

,class_6,class_15,class_16,class_42,class_52,class_53,class_62,class_64,class_65,class_67,class_88,class_90,class_92,class_95,class_99,object_id
3492885,0.000111,0.307268,0.000201,0.257094,0.195776,0.000226,0.058261,0.000083,0.001280,0.010093,0.003744,0.165445,0.000109,0.000309,0.246052,130787966
3492886,0.000158,0.016633,0.000169,0.075519,0.214118,0.000225,0.017595,0.226793,0.000347,0.033160,0.000673,0.410798,0.000161,0.003651,0.226427,130787971
3492887,0.000545,0.025822,0.000116,0.818583,0.015634,0.000317,0.120560,0.000038,0.000176,0.001060,0.000456,0.015168,0.000146,0.001380,0.121208,130787974
3492888,0.000043,0.781937,0.000066,0.112278,0.005420,0.000063,0.009198,0.053842,0.000205,0.000444,0.000112,0.035626,0.000082,0.000685,0.137913,130788053
3492889,0.000087,0.054270,0.000093,0.377521,0.265302,0.000188,0.042212,0.000241,0.000232,0.008590,0.001412,0.249463,0.000100,0.000288,0.229221,130788054


In [13]:
test_ensemble = test_ensemble[list(pred1.columns)]

In [14]:
test_ensemble.tail()

,object_id,class_6,class_15,class_16,class_42,class_52,class_53,class_62,class_64,class_65,class_67,class_88,class_90,class_92,class_95,class_99
3492885,130787966,0.000111,0.307268,0.000201,0.257094,0.195776,0.000226,0.058261,0.000083,0.001280,0.010093,0.003744,0.165445,0.000109,0.000309,0.246052
3492886,130787971,0.000158,0.016633,0.000169,0.075519,0.214118,0.000225,0.017595,0.226793,0.000347,0.033160,0.000673,0.410798,0.000161,0.003651,0.226427
3492887,130787974,0.000545,0.025822,0.000116,0.818583,0.015634,0.000317,0.120560,0.000038,0.000176,0.001060,0.000456,0.015168,0.000146,0.001380,0.121208
3492888,130788053,0.000043,0.781937,0.000066,0.112278,0.005420,0.000063,0.009198,0.053842,0.000205,0.000444,0.000112,0.035626,0.000082,0.000685,0.137913
3492889,130788054,0.000087,0.054270,0.000093,0.377521,0.265302,0.000188,0.042212,0.000241,0.000232,0.008590,0.001412,0.249463,0.000100,0.000288,0.229221


In [15]:
test_ensemble['object_id'].equals(pred1['object_id'])

True

In [16]:
%%time
test_ensemble.to_csv('test_pred_23.csv',index=False)

CPU times: user 1min 32s, sys: 1.37 s, total: 1min 33s
Wall time: 1min 34s


In [17]:
!kaggle competitions submit -c PLAsTiCC-2018 -f test_pred_23.csv -m "Message"

100%|█████████████████████████████████████| 1.06G/1.06G [2:37:51<00:00, 121kB/s]
Successfully submitted to PLAsTiCC Astronomical Classification